In [ ]:
import numpy as np
from skimage import io

import napari
import pandas as pd

import zarr

import os
import pickle

### Make data for training
Assume training data is double color. One for the channel and another for Cytosol/nuclei.

In [ ]:
# image path
fix_n5_path = '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/fused/fused.n5' # zarr with pyramid resolution
# save path
save_path = '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/model_training/crops'
# metadata path
meta_path = '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/model_training/info.pkl'

# create Zarr file object
fix_zarr = zarr.open(store=zarr.N5Store(fix_n5_path), mode='r')
# if you use ngff ome.zarr
# mov_zarr_path = '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/registration/round02.zarr'
# mov_zarr = zarr.open(mov_zarr_path, mode='r')

n5_setups = list(fix_zarr.keys())

voxel_size = (2.0,1.3,1.3)

In [ ]:
# set your parameters here
reference_chan = 3 # Number or None
segment_chan = 2

# set corner positions. I suggest finding the positions using BigDataViewer or relevant.
corner_positions = [
    [1235,2510,776],
    [1325,3350,1543],
    [1725,4632,1604],
    [2262,6207,2682],
    [1708,3877,2563]
]

# [100,256,256] crop size and FoV [100,768,768] are recommended for the 2D annotation
crop_size = [100,256,256]
FoV = [100,768,768]

# set True for 2D annotation and set False for 3D annotation
select_plane = True

# processing of parameters
if not all([(j-i)>=0 for i,j in zip(crop_size, FoV)]):
    raise ValueError('FoV should be larger than crop_size')

In [ ]:
# make metadata file if it does not exist.
if not os.path.exists(meta_path):
    df = pd.DataFrame(columns=['ID', 'integer_ID', 'instance_counts', 'corner', 'source', 'ref_channel', 'channel', 'crop_size'])
else:
    df = pd.read_pickle(meta_path)

for i, pos in enumerate(corner_positions):
    print(f"The index {i} with the position {pos}")
    idx = len(df)
    
    # find out any duplication between the current data and the metadata
    # if it is duplicated, ask 
    flag = False
    if df['corner'].isin([pos]).any():
        for k in df['integer_ID'][df['corner'].isin([pos])].to_list():
            if ((df.loc[k,'source'] == fix_n5_path) and 
                (df.loc[k,'ref_channel'] == reference_chan) and 
                (df.loc[k,'channel'] == segment_chan) and 
                (df.loc[k,'crop_size'] == crop_size) and
                (df.loc[k,'select_plane'] == select_plane)):
                flag = True
                idx = k
    if flag:
        ans = input("Do you want to re-analyze the data? y or n")
        if ans != 'y':
            continue
        
    # set file path to be saved for both image and mask
    prefix = str(idx)
    while len(prefix) < 4:
        prefix = '0' + prefix
    img_path = os.path.join(save_path, prefix+'_img.tif')
    mask_path = os.path.join(save_path, prefix+'_mask.tif')



    # get the image of a channel to be segmented
    FoV_stack = []
    img = fix_zarr[n5_setups[segment_chan]]['timepoint0']['s0']

    # set the corner of FoV in napari
    top_corner = tuple(i-(k-j)//2 for i,j,k in zip(pos, crop_size, FoV))
    bottom_corner = tuple(i+j+(k-j)//2 for i,j,k in zip(pos, crop_size, FoV))
    top_corner = tuple(j if j>=i else i for i,j in zip([0,0,0],top_corner))
    bottom_corner = tuple(j if j<=i else i for i,j in zip(img.shape,bottom_corner))
    
    # prepare to make border lines
    top_border_corner = tuple((k-j)//2 for j,k in zip(crop_size, FoV))
    bottom_border_corner = tuple(j+(k-j)//2 for j,k in zip(crop_size, FoV))
    
    FoV_segment = img[tuple(slice(i,j) for i,j in zip(top_corner, bottom_corner))]
    
    # get the image of a reference of channel
    if reference_chan is not None:
        img = fix_zarr[n5_setups[reference_chan]]['timepoint0']['s0']
        FoV_reference = img[tuple(slice(i,j) for i,j in zip(top_corner, bottom_corner))]
        FoV_stack.append(FoV_reference)
    
    FoV_stack.append(FoV_segment)
    FoV_stack = np.stack(FoV_stack)

    # open Napari. Pause for loop until close the window
    viewer = napari.Viewer()
    viewer.add_image(FoV_stack, channel_axis=0, scale=voxel_size, contrast_limits=[0,65535])
    viewer.add_shapes([[bottom_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]],[top_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]]],
                      edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer.add_shapes([[top_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]],[top_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer.add_shapes([[bottom_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]],[bottom_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer.add_shapes([[bottom_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]],[top_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line')
    labels = viewer.add_labels(np.zeros_like(FoV_segment), name='segmentation', scale=voxel_size)
    viewer.show(block=True)
    
    sub_area_slicer = tuple(slice(i,j) for i,j in zip(top_border_corner,bottom_border_corner))

    # save images and segmentation.
    img = np.swapaxes(FoV_stack[(slice(0,None),)+sub_area_slicer],0,1)
    labels_img = labels.data[sub_area_slicer]
    
    if select_plane:
        plane_pos = np.argmax((labels.data>0).sum(axis=(1,2)))
        img = img[plane_pos,...]
        labels_img = labels_img[plane_pos,...]
        
        io.imsave(img_path, img, plugin='tifffile', imagej=True, metadata={'axes': 'CYX'})
        io.imsave(mask_path, labels_img, plugin='tifffile')

    else:
        io.imsave(img_path, img, plugin='tifffile', imagej=True, metadata={'axes': 'ZCYX'})
        io.imsave(mask_path, labels_img, plugin='tifffile')

    # update the metadata
    df.loc[idx,'ID'] = prefix
    df.loc[idx,'integer_ID'] = idx
    count = (np.unique(labels_img)).size - 1
    df.loc[idx,'instance_counts'] = count
    df.at[idx,'corner'] = pos
    df.loc[idx, 'source'] = fix_n5_path
    df.loc[idx, 'ref_channel'] = reference_chan
    df.loc[idx, 'channel'] = segment_chan
    df.at[idx, 'crop_size'] = crop_size
    df.loc[idx, 'select_plane'] = select_plane

# pickle the metadata
df.to_pickle(meta_path)